In [1]:
# [데이터 출처]
# 전국주차장정보표준데이터
# https://www.data.go.kr/data/15012896/standard.do

In [2]:
import pandas as pd

parkinglot = pd.read_csv('parkinglot.csv', encoding='cp949')

# 필요 없는 컬럼 삭제
cols_to_drop = parkinglot.columns.difference(['주차장명', '위도', '경도'])
parkinglot = parkinglot.drop(cols_to_drop, axis=1)
parkinglot

C:\Users\chang\AppData\Local\Temp\ipykernel_14556\1388093372.py:3: DtypeWarning: Columns (20,23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  parkinglot = pd.read_csv('parkinglot.csv', encoding='cp949')


,주차장명,위도,경도
0,횡계의원 뒤 1주차장,37.671701,128.707965
1,횡계의원 뒤 2주차장,37.671447,128.707964
2,오삼불고기거리 주차장,37.672446,128.708479
3,하이랜드 옆 1주차장,37.673161,128.709291
4,하이랜드 옆 2주차장,37.673531,128.709376
...,...,...,...
16927,행운주차장,NaN,NaN
16928,향토주차장,NaN,NaN
16929,현대주차장,35.162277,129.159375
16930,흥안인더스트리,NaN,NaN


In [3]:
# 위도 경도 결측치를 포함한 행 제거
parkinglot = parkinglot.dropna()
parkinglot.shape

(13849, 3)

In [4]:
import pymysql
from dotenv import load_dotenv
import os

# DB 비밀번호 불러오기
load_dotenv()
db_password = os.getenv("DB_PASSWORD")

# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='root' , password=db_password, db='parkinglot', charset='utf8')

In [19]:
# Cursor 객체 생성
cursor = conn.cursor()

# 단순 double 자료형 저장 테이블 생성
cursor.execute("DROP TABLE IF EXISTS parkinglot_double")

double_table_query = """
CREATE TABLE parkinglot_double (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(200),
    latitude DOUBLE,
    longitude DOUBLE
)
"""
cursor.execute(double_table_query)
conn.commit()


# point 자료형 저장 테이블 생성
cursor.execute("DROP TABLE IF EXISTS parkinglot_point")

point_table_query = """
CREATE TABLE parkinglot_point (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(200),
    location POINT NOT NULL
)
"""
cursor.execute(point_table_query)
conn.commit()


# point 자료형 with index 저장 테이블 생성
cursor.execute("DROP TABLE IF EXISTS parkinglot_pointindex")

pointindex_table_query = """
CREATE TABLE parkinglot_pointindex (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(200),
    location POINT NOT NULL,
    spatial index (location)
)
"""
cursor.execute(pointindex_table_query)
conn.commit()


In [20]:
# 데이터 저장
for _ in range(10):
    for index, row in parkinglot.iterrows():
        insert_data_query = f"""
        INSERT INTO parkinglot_double (name, latitude, longitude) 
        VALUES ('{row['주차장명']}', {row['위도']}, {row['경도']})
        """
        cursor.execute(insert_data_query)

        insert_data_query = f"""
        INSERT INTO parkinglot_point (name, location) 
        VALUES ('{row['주차장명']}', point({row['경도']}, {row['위도']}))
        """
        cursor.execute(insert_data_query)

        insert_data_query = f"""
        INSERT INTO parkinglot_pointindex (name, location) 
        VALUES ('{row['주차장명']}', point({row['경도']}, {row['위도']}))
        """
        cursor.execute(insert_data_query)

    conn.commit()
        

# 테스트 쿼리

<pre>
SELECT 
id, name,
(
  6371 *
  acos(
    cos(radians(36.628486474734 /* 중앙도서관의 위도 */)) * 
    cos(radians(latitude)) * 
    cos(radians(longitude) - 
        radians(127.4574415007155 /* 중앙도서관의 경도 */)) + 
    sin(radians(36.628486474734 /* 중앙도서관의 위도 */)) * 
    sin(radians(latitude)))
) AS distance 
FROM parkinglot_double
HAVING DISTANCE < 50
ORDER BY distance;
</pre>

<pre>
SELECT 
id, name, ST_DISTANCE_SPHERE(location, POINT(127.4574415007155, 36.628486474734)) AS distance 
FROM parkinglot_point
HAVING DISTANCE < 50 * 1000
ORDER BY distance;
</pre>

<pre>
SELECT 
id, name, ST_DISTANCE_SPHERE(location, POINT(127.4574415007155, 36.628486474734)) AS distance 
FROM parkinglot_pointindex
HAVING DISTANCE < 50 * 1000
ORDER BY distance;
</pre>

# 쿼리 실행 프로파일링 결과

- 1번 쿼리 (단순 double 자료형 저장) : 0.0509045
- 2번 쿼리 (point 자료형 저장) : 0.1008433
- 3번 쿼리 (point 자료형 with index 저장) : 0.0877484